In [14]:
import sys
!{sys.executable} -m pip install rake_nltk
import sys
!{sys.executable} -m pip install fuzzywuzzy
import jieba
import gensim
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import math
import re
from rake_nltk import Rake
from collections import Counter
from gensim import corpora, models, similarities
lemmer = nltk.stem.WordNetLemmatizer()
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
class RakeImpl:
 
    def __init__(self, text):
        self.text = text
        self.rake = Rake()
 
    def getKeywords(self):
        self.rake.extract_keywords_from_text(self.text)
        return self.rake.get_ranked_phrases()

In [27]:
class Eval:
    def __init__(self,doc1,doc2,fullMarks):
        self.doc1 = ' '.join(self.LemNormalize(doc1))
        self.doc2 = doc2
        self.fullMarks = fullMarks
        self.marks = self.weightedSum()*self.fullMarks
       
        
    def LemTokens(self,tokens):
        return [lemmer.lemmatize(token) for token in tokens]
    
    def LemNormalize(self,text):
        remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
        return self.LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    
    
    def Jaccard_Similarity(self): 
        doc1 = self.doc1
        doc2 = self.doc2
        # List the unique words in a document
        words_doc1 = set(doc1.lower().split()) 
        words_doc2 = set(doc2.lower().split())
    
        # Find the intersection of words list of doc1 & doc2
        intersection = words_doc1.intersection(words_doc2)

        # Find the union of words list of doc1 & doc2
        union = words_doc1.union(words_doc2)
        
        # Calculate Jaccard similarity score 
        # using length of intersection set divided by length of union set
        return float(len(intersection)) / len(union)

    def text_to_vector(self,text):
        WORD = re.compile(r"\w+")
        words = WORD.findall(text)
        return Counter(words)
    
    def get_cosine(self,vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
        sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        if not denominator:
            return 0.0
        else:
            return float(numerator) / denominator
        
    def cosSimilarity(self):
        keyword_inst1 = RakeImpl(self.doc1)
        keyword_inst2 = RakeImpl(self.doc2)
        v1 = keyword_inst1.getKeywords()
        v2 = keyword_inst2.getKeywords()
        str2 = ' '.join(v2)
        str1 = ' '.join(v1)
        vector1 = self.text_to_vector(str1)
        vector2 = self.text_to_vector(str2)
        x = self.get_cosine(vector1,vector2)
        return x
    
    def bigramSimilarity(self):
        tokens1 = nltk.word_tokenize(self.doc1)
        tokens2 = nltk.word_tokenize(self.doc2)
        bigrams1 = nltk.bigrams(tokens1)
        bigrams2 = nltk.bigrams(tokens2)
        frequency1 = nltk.FreqDist(bigrams1)
        frequency2 = nltk.FreqDist(bigrams2)
        count=0
        for key in frequency1.items():
            if key in frequency2.items():
                count=count+1
        l1=len(frequency1)
        l2=len(frequency2)
        avg=(l1+l2)/2
        similarity=count/avg
        return similarity       
    
    def fuzzyMarks(self):
        return fuzz.ratio(self.doc1, self.doc2)
    def weightedSum(self):
        return (0.2*self.cosSimilarity() + 0.006*self.fuzzyMarks()+0.2*self.bigramSimilarity())*self.fullMarks 
    
    def printMarks(self):
        print("Marks: ",self.marks,"/",self.fullMarks)

In [28]:
doc1 = "LeBron James is a basketball legend who is not better than Michael Jordan - says Charles Barkley"
doc2 = "Kobe Bryant is the better Than LeBron James as a basketball player"
fullMarks = 2
similarity = Eval(doc1,doc2,fullMarks)

In [29]:
similarity.printMarks()

Marks:  1.3080116165254674 / 2


In [8]:
similarity.cosSimilarity()


0.47809144373375745

In [9]:
import nltk, string, numpy
nltk.download('punkt') # first-time use only
stemmer = nltk.stem.porter.PorterStemmer() 
def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def StemNormalize(text):
    return StemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
StemNormalize(doc1)

['lebron',
 'jame',
 'is',
 'a',
 'basketbal',
 'legend',
 'who',
 'is',
 'not',
 'better',
 'than',
 'michael',
 'jordan',
 'say',
 'charl',
 'barkley']

In [11]:
lemmer = nltk.stem.WordNetLemmatizer()
import string
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [12]:
LemNormalize(doc1)

['lebron',
 'james',
 'is',
 'a',
 'basketball',
 'legend',
 'who',
 'is',
 'not',
 'better',
 'than',
 'michael',
 'jordan',
 'say',
 'charles',
 'barkley']

In [15]:
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

C:\Users\Asus\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
